# Detecting Kermit and Waldorf & Statler based on audio features

This notebook documents our approach to detect Kermit and Waldorf and Statler based on audio feature-engineering.  
We employ a Logistic Regression Classifier to predict the characters based on different audio-engineering features used for the different characters.

## Time sheet for this notebook

**Daniel Blasko:**
| Date | Task | Hours |
| --- | --- | --- |
| 27.11.2023 | Setup notebook, first experiments | 3 |
| 27.11.2023 | Implement "utils/MuppetDataset.py" that generally loads and handles the annotated video data | 1 |
| 28.11.2023 | Experiment & build feature extraction for both characters, align audio samples with frame annotations | 2 |


## Imports


In [1]:
import matplotlib.pyplot as plt
import librosa.feature as lf
import librosa
import numpy as np

import sys

sys.path.append("..")
from utils.MuppetDataset import MuppetDataset


## Loading the data

Set the booleans below to extract the audio/frames from the .avi files if it has not been done previously.


In [2]:
extract_audio = True
extract_frames = True


In [ ]:
video_paths = [
    "../data/Muppets-02-01-01.avi",
    "../data/Muppets-02-04-04.avi",
    "../data/Muppets-03-04-03.avi",
]
annotation_paths = [
    "../data/GroundTruth_Muppets-02-01-01.csv",
    "../data/GroundTruth_Muppets-02-04-04.csv",
    "../data/GroundTruth_Muppets-03-04-03.csv",
]

dataset = MuppetDataset(video_paths, annotation_paths, extract_audio, extract_frames)


Example for handling the data for video 0:

```python
dataset.audio_paths[0]
dataset.audios[0]
dataset.annotations.loc[dataset.annotations.Video == 0]
```


## Audio feature extraction


### Aligning audio features with video frame rate

The annotations are at the video frame level, for audio too. Therefore, we need to align the audio features with the video frames.

We start by checking the framerate of the videos and remind of our audio sampling rate:


In [5]:
%%sh
ffprobe -v error -select_streams v:0 -show_entries stream=avg_frame_rate -of default=noprint_wrappers=1:nokey=1 ../data/Muppets-02-01-01.avi


25/1


In [11]:
FRAMES_PER_SECOND = 25


In [19]:
AUDIO_SAMPLING_RATE = dataset.audios[0]["sr"]
AUDIO_SAMPLING_RATE


44100

There are 25 frames per second, and the 16k audio samples per second.  
We therefore have $\frac{44100}{25} = 1764$ audio samples per frame and divide our audio features in windows of 1764 samples.


In [ ]:
video_length_in_frames = dataset.annotations.loc[
    dataset.annotations.Video == 0
].Frame_number.max()


video_duration_seconds = video_length_in_frames / FRAMES_PER_SECOND
required_audio_length = int(video_duration_seconds * AUDIO_SAMPLING_RATE)


### Features for Kermit

We make our decisions based on the observation that Kermit displays a distinct audio pattern where his interventions start with screaming, and transition to mumbling as he speaks, which should correspond to a high foundational frequency.  
We therefore decide to extract the fundational frequency of the audio (pitch), as well as loudness.

**We normalize all extracted features.**


In [29]:
for audio in dataset.audios:
    # Pad audio with silence to extract features from the last 8 frames
    audio["audio"] = np.pad(
        audio["audio"],
        (0, required_audio_length - dataset.audios[0]["audio"].shape[0]),
        "constant",
    )
    # Loudness (through RMS energy)
    audio["loudness_rms"] = librosa.util.normalize(
        librosa.feature.rms(
            y=audio["audio"], hop_length=int(audio["sr"] / FRAMES_PER_SECOND)
        )[0]
    )
    # Zero crossing rate
    audio["zcr"] = librosa.util.normalize(
        librosa.feature.zero_crossing_rate(
            y=audio["audio"], hop_length=int(audio["sr"] / FRAMES_PER_SECOND)
        )[0]
    )


This approach with the `hop_length` of 1764 samples leads to feature vectors of the length of the number of frames, which is what we desired.


### Features for Waldorf & Statler
